# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weather_data.csv")
weather_data

,city,lat,lon,temp,temp_max,windspeed,cloudall,humidity
0,rikitea,-23.1203,-134.9692,74.12,74.12,16.26,9,80
1,jamestown,42.0970,-79.2353,62.51,65.52,7.00,90,95
2,east london,-33.0153,27.9116,68.94,68.94,28.77,20,60
3,lebu,-37.6167,-73.6500,54.12,54.12,12.73,13,80
4,busselton,-33.6500,115.3333,50.04,51.08,12.12,1,74
...,...,...,...,...,...,...,...,...
545,taywarah,33.3500,64.4167,56.08,56.08,10.04,0,15
546,cacoal,-11.4386,-61.4472,88.43,88.43,3.67,100,53
547,gayny,60.3071,54.3241,40.21,40.21,5.73,100,84
548,mehamn,71.0357,27.8492,36.32,36.32,5.79,100,94


In [3]:
### Humidity Heatmap
#Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.


# Need to get geocodes for citites and count the resturants per city as the degree for the heat map 
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_data[["lat", "lon"]]
humidity = weather_data["humidity"].astype(float)


In [48]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 90 degrees but higher than 75.
* Wind speed less than 10 mph.
* < 5% cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [35]:
# Using the c
ideal_weather_data = weather_data.loc[(weather_data["temp_max"] < 90) & 
                                      (weather_data["temp_max"] > 75) & 
                                      (weather_data["cloudall"] < 5) & 
                                      (weather_data["windspeed"] < 10)] 

ideal_weather_data = ideal_weather_data.reindex(columns = ideal_weather_data.columns.tolist() 
                                  + ["Hotel Name"])
ideal_weather_data 


,city,lat,lon,temp,temp_max,windspeed,cloudall,humidity,Hotel Name
134,lumding,25.7535,93.1732,78.03,78.03,0.83,2,91,NaN
139,la roda,39.2167,-2.1500,72.39,76.91,5.14,0,42,NaN
178,puerto escondido,15.8500,-97.0667,76.03,76.03,5.75,1,73,NaN
181,myitkyina,25.3833,97.4000,79.92,79.92,0.20,0,77,NaN
225,dzaoudzi,-12.7887,45.2699,85.89,85.89,8.05,0,58,NaN
276,bani,18.2833,-70.3333,80.17,82.85,3.65,0,68,NaN
288,upington,-28.4478,21.2561,79.18,79.18,9.22,0,18,NaN
323,tezu,27.9167,96.1667,77.49,77.49,3.11,0,84,NaN
340,jabinyanah,35.0350,10.9081,80.40,80.40,9.04,0,35,NaN
432,gilgit,35.9221,74.3087,75.36,75.36,3.51,0,22,NaN


### Hotel Map
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
target_type = "hotel"
radius = 5000

# Loop through the dataframe, call the google api for each row and find the name of the Hotel near by
# add the hotel name to the ideal_weather_data dataframe 

for index,row in ideal_weather_data.iterrows():
    coords = str(row["lat"]) + ","+ str(row["lon"])

    params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": g_key
    }

    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:
        # Run request
        response = requests.get(base_url, params)

        # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
        #print(response.url)
    
        hotels = response.json()
        print(index)
        print(hotels["results"][0]["name"])
        ideal_weather_data[index,"Hotel Name"] = hotels["results"][0]["name"]
        
    except:
        print("No hotels in the area") 

ideal_weather_data
        
#ideal_weather_data["Hotel Name"] 


134
Lumding
139
La Roda
178
Puerto Escondido
181
Myitkyina
225
Mamoudzou
276
Baní
288
Upington
323
Tezu
340
Jebiniana
432
Gilgit
500
İdil
535
Santa Marta


,city,lat,lon,temp,temp_max,windspeed,cloudall,humidity,Hotel Name,"(134, Hotel Name)",...,"(178, Hotel Name)","(181, Hotel Name)","(225, Hotel Name)","(276, Hotel Name)","(288, Hotel Name)","(323, Hotel Name)","(340, Hotel Name)","(432, Hotel Name)","(500, Hotel Name)","(535, Hotel Name)"
134,lumding,25.7535,93.1732,78.03,78.03,0.83,2,91,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
139,la roda,39.2167,-2.1500,72.39,76.91,5.14,0,42,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
178,puerto escondido,15.8500,-97.0667,76.03,76.03,5.75,1,73,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
181,myitkyina,25.3833,97.4000,79.92,79.92,0.20,0,77,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
225,dzaoudzi,-12.7887,45.2699,85.89,85.89,8.05,0,58,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
276,bani,18.2833,-70.3333,80.17,82.85,3.65,0,68,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
288,upington,-28.4478,21.2561,79.18,79.18,9.22,0,18,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
323,tezu,27.9167,96.1667,77.49,77.49,3.11,0,84,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
340,jabinyanah,35.0350,10.9081,80.40,80.40,9.04,0,35,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta
432,gilgit,35.9221,74.3087,75.36,75.36,3.51,0,22,NaN,Lumding,...,Puerto Escondido,Myitkyina,Mamoudzou,Baní,Upington,Tezu,Jebiniana,Gilgit,İdil,Santa Marta


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_data.iterrows()]


In [43]:
hotel_info

['\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>lumding</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>la roda</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>puerto escondido</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>myitkyina</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>dzaoudzi</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>bani</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>upington</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>tezu</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>jabinyanah</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>gilgit</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>idil</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel</dt><dd>nan</dd>\n<dt>City</dt><dd>santa marta</dd>\n</dl>\n']

In [49]:
# Add marker layer ontop of heat map
marker_locations = ideal_weather_data[['lat', 'lon']]

# Create a marker_layer using the ideal weather info 
#fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=[info_box_template.format(**row) for index, row in ideal_weather_data.iterrows()])
# Display figure

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))